## 题目：寻找关键用户和分析社交关系

### data.json 数据描述
#### 1. 用户节点数据包含用户ID和他们的"影响力分数"，格式为["用户名称", 影响力分数]。
#### 2. 边关系数据包含每两个用户之间的连接强度（即边的权重），格式为["用户名称1", "用户名称2", 关系权重]。
例如：
- 节点：["Yanyan-Wang", 50.95], ["Aarebecca", 23.82], ...
- 边：["Yanyan-Wang", "Aarebecca", 25.3], ["Yanyan-Wang", "pomelo-nwu", 25.09], ...

### 问题
#### 1. 社交网络构建与分析
- a. 使用邻接矩阵或者字典表示法来表示这个社交关系网络。
- b. 请找出每个用户的直接朋友（即与其存在边关系的节点），并用列表形式列出。

#### 2. 统计与度中心性分析
- a. 计算每个用户的度数（即直接朋友数量），列出度数最多的前5个用户及其朋友数量。
- b. 通过影响力分数（如上所示），找出影响力最高的前5个用户。
- c. 综合考虑影响力分数和朋友数量，列出对社交网络最有影响力的用户（可以选择将两者按一定比例加权，例如影响力得分的50%和朋友数量的50%）。

#### 3. 推荐新朋友
- a. 对于用户 Yanyan-Wang，请找出其“朋友的朋友”中最有可能成为新朋友的用户，并列出推荐的理由（例如有最多共同的朋友）。
- b. 考虑用户之间的连接强度，找出与 Yanyan-Wang 连接强度最高的5个用户（无论是否已经是直接朋友），并列出推荐排序。
 
#### 4. 网络图的可视化
- a. 使用图论可视化工具绘制用户关系网络图，展示关键用户和他们的连接情况。
- b. 你可以对图中的节点按“影响力得分”进行着色，节点的大小则代表度数，尝试解释在图中哪些用户在社交网络中最具影响力。

In [ ]:
import json

with open('hw4_data.json','r') as file:
    data = json.load(file)
    
nodes = data['nodes']
edges = data['edges']

#1a
dict = {}

for e in edges:
    dict[e[0]+ ' and '+e[1]] = e[2]
    
print(dict)
    

#1b
friends = {}
for n in nodes:
    friends[n[0]] = []
    
for e in edges:
    friends[e[0]].append(e[1])
    friends[e[1]].append(e[0])
    
for key,value in friends.items():
    print (key,value)

In [ ]:
import json 

with open('hw4_data.json','r') as file:
    data = json.load(file)
    
people = data['nodes']
edges = data['edges']


#2a
dict = {}
for p in people:
    dict[p[0]] = []
    
for e in edges:
    dict[e[0]].append(e[1])
    dict[e[1]].append(e[0])
    
degree = []
for key,value in dict.items():
    degree.append([key,len(value)])
    
sorted_degree = sorted(degree,key=lambda x:x[1],reverse=True)
print('people with most friends:')
for i in range(5):
    print(sorted_degree[i])
    
    
#2b
sorted_people = sorted(people,key=lambda x:x[1],reverse=True)
print('people with highest raw influnce score:')
for i in range(5):
    print(sorted_people[i])
    
#2c
dict_comprehensive = {}
for p in people:
    dict_comprehensive[p[0]] = p[1] * 0.5
for e in edges:
    dict_comprehensive[e[0]] = dict_comprehensive[e[0]] + 0.5
    dict_comprehensive[e[1]] = dict_comprehensive[e[1]] + 0.5
    

list_comprehensive = []
for key,value in dict_comprehensive.items():
    list_comprehensive.append([key,value])
    
sorted_comprehensive = sorted(list_comprehensive,key=lambda x:x[1],reverse=True)
print("people with highest comprehensive influence score:(friends * 0.5 + raw influence * 0.5)")
for i in range(5):
    print(sorted_comprehensive[i])
    




In [ ]:
import json
import queue

with open('hw4_data.json','r') as file:
    data = json.load(file)
    
people = data['nodes']
edges = data['edges']

layer_dict = {}  #表示朋友的层级，yanyanwang为0
friends_dict = {}
connection_dict = {}

for p in people:
    layer_dict[p[0]] = 9999
    friends_dict[p[0]] = []
    
for e in edges:
    friends_dict[e[0]].append(e[1])
    friends_dict[e[1]].append(e[0])
    
for e in edges:
    connection_dict[e[0]+e[1]] = e[2]
    connection_dict[e[1]+e[0]] = e[2]
    

    
    
    
    
    
#3a
f2_to_f1 = {}  #衡量yanyanwang的二级朋友和一级朋友数的总连接强度

q = queue.Queue()
q.put('Yanyan-Wang')
layer_dict['Yanyan-Wang'] = 0

while q.empty() == False:
    head_name = q.get()
    head_layer = layer_dict[head_name]
    
    if head_layer == 2:
        f2_to_f1[head_name] = 0
    
    
    for fr in friends_dict[head_name]:
        if layer_dict[fr] == 9999:
            q.put(fr)
            layer_dict[fr] = head_layer + 1
        
        if head_layer == 2 and layer_dict[fr] == 1:
            f2_to_f1[head_name] = f2_to_f1[head_name] + connection_dict[head_name+fr]
            


#for key,value in f2_to_f1.items():
#   print(key,value)

best_friend = ""
highest_connection = 0
for key,value in f2_to_f1.items():
    if value > highest_connection:
        best_friend = key
        highest_connection = value

print('recommended friend:',end="")
print(best_friend)
    
    
#3b
friend_list = []
for f in friends_dict['Yanyan-Wang']:
    friend_list.append([f,connection_dict[f+"Yanyan-Wang"]])
    
sorted_list = sorted(friend_list,key=lambda x:x[1],reverse=True)
for i in range(5):
    print(sorted_list[i])
    


    

In [1]:
import matplotlib.pyplot as plt
import networkx as nx
import json

with open('hw4_data.json','r') as file:
    data = json.load(file)
people = data['nodes']
edges = data['edges']



friends_dict = {}
for p in people:
    friends_dict[p[0]] = 0
    
for e in edges:
    friends_dict[e[0]] = friends_dict[e[0]] + 1
    friends_dict[e[1]] = friends_dict[e[1]] + 1
    
    



#4a
plt.figure(figsize=(10,10))
G = nx.Graph()
for p in people:
    G.add_node(p[0],color = 'green')
for e in edges:
    G.add_edge(e[0],e[1],weight = e[2])
    
    
color = []
for p in people:
    if p[1] > 15:
        color.append('red')
    elif p[1] > 6:
        color.append('yellow')
    elif p[1] > 3:
        color.append('blue')
    else:
        color.append('green')    
        
size = []
for p in people:
    if friends_dict[p[0]] > 20:
        size.append(1000)
    elif friends_dict[p[0]] > 15:
        size.append(600)
    elif friends_dict[p[0]] > 10:
        size.append(400)
    else:
        size.append(100)








nx.draw(G,with_labels=True,node_color = color,node_size = size,edge_color = 'grey')
plt.subplots_adjust(left=0.15, right=0.2, bottom=0.1, top=0.2) 
plt.show()


分析哪些用户在社交网络上最具影响力：
pomelo-nwu,frank-zsy 和 tyn1998 最具影响力，因为他们的节点颜色是红色，节点尺寸最大。这代表他们有最多的直接朋友和最大的影响力。